In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [ ]:
!apt-get -y install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java
  libatk-wrapper-java-jni libbabeltrace1 libcub-dev libcublas11 libcublaslt11
  libcudart11.0 libcufft10 libcufftw10 libcuinj64-11.5 libcupti-dev
  libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1
  libgail-common libgail18 libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglvnd-core-dev libglvnd-dev libglx-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11
  libnpps11 libnvblas11 libnvidia-compute-495 libnvidia-compute-510
  libnvidia-compute-525 libnvidia-ml-dev libnvjpeg11 libnvrtc-builtins11.5
  libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-de

In [ ]:
!nvidia-smi

Sun Sep 24 10:33:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wdh5vxv3
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wdh5vxv3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=3c34dd71e3c9cf72eab2c84c5ab5dcf1f9d83aead02f41f5f32e288bf349e1a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ww3g8id/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <time.h>

#define CSC(call) 							\
do { 										\
	cudaError_t status = call;				\
	if (status != cudaSuccess) {																				\
		fprintf(stderr, "ERROR is %s:%d. Message: %s\n", __FILE__, __LINE__, cudaGetErrorString(status));		\
		exit(0);								\
	}											\
} while(0)

/*
    threadIdx.x, threadIdx.y, threadIdx.z
    blockIdx.x, blockIdx.y, blockIdx.z
    blockDim.x, blockDim.y, blockDim.z,
    gridDim.x, gridDim.y, gridDim.z,
*/
// __device__ __host__


__global__ void kernel(double* arr1, double* arr2, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int offset = gridDim.x * blockDim.x;
    while(idx < n) {
        arr1[idx] = min(arr1[idx], arr2[idx]);
        idx += offset;
    }
}


int main() {
    int n;
    n = 1000;
    double* arr1 = (double*)malloc(sizeof(double) * n);
    for(int i = 0; i < n; i++) {
        arr1[i] = i;
    }
    double* arr2 = (double*)malloc(sizeof(double) * n);
    for(int i = 0; i < n; i++) {
        arr2[i] = i;
    }

    double* device_arr1;
    cudaMalloc(&device_arr1, sizeof(double) * n);
    cudaMemcpy(device_arr1, arr1, sizeof(double) * n, cudaMemcpyHostToDevice);
    double* device_arr2;
    cudaMalloc(&device_arr2, sizeof(double) * n);
    cudaMemcpy(device_arr2, arr2, sizeof(double) * n, cudaMemcpyHostToDevice);
    cudaEvent_t start, stop;
	  CSC(cudaEventCreate(&start));
	  CSC(cudaEventCreate(&stop));
	  CSC(cudaEventRecord(start));

    kernel<<<1,32>>>(device_arr1, device_arr2, n);
    CSC(cudaDeviceSynchronize());
	  CSC(cudaGetLastError());

	  CSC(cudaEventRecord(stop));
	  CSC(cudaEventSynchronize(stop));
	  float t;
	  CSC(cudaEventElapsedTime(&t, start, stop));
	  CSC(cudaEventDestroy(start));
	  CSC(cudaEventDestroy(stop));

	  printf("time = %f ms\n", t);

    cudaMemcpy(arr1, device_arr1, sizeof(double) * n, cudaMemcpyDeviceToHost);

    for(int i = 0; i < n; i++) {
        printf("%f ", arr1[i]);
    }
    printf("\n");

    free(arr1);
    cudaFree(device_arr1);
    free(arr2);
    cudaFree(device_arr2);
    return 0;
}

time = 0.059776 ms
0.000000 1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 18.000000 19.000000 20.000000 21.000000 22.000000 23.000000 24.000000 25.000000 26.000000 27.000000 28.000000 29.000000 30.000000 31.000000 32.000000 33.000000 34.000000 35.000000 36.000000 37.000000 38.000000 39.000000 40.000000 41.000000 42.000000 43.000000 44.000000 45.000000 46.000000 47.000000 48.000000 49.000000 50.000000 51.000000 52.000000 53.000000 54.000000 55.000000 56.000000 57.000000 58.000000 59.000000 60.000000 61.000000 62.000000 63.000000 64.000000 65.000000 66.000000 67.000000 68.000000 69.000000 70.000000 71.000000 72.000000 73.000000 74.000000 75.000000 76.000000 77.000000 78.000000 79.000000 80.000000 81.000000 82.000000 83.000000 84.000000 85.000000 86.000000 87.000000 88.000000 89.000000 90.000000 91.000000 92.000000 93.000000 94.000000 95.000000 96.000000 97.000000 98.000000 9

In [ ]:
%%cu
#include <stdio.h>

int main() {
	int deviceCount;
	cudaDeviceProp devProp;
	cudaGetDeviceCount(&deviceCount);
	printf("Found %d devices\n", deviceCount);
	for(int device = 0;device < deviceCount;device++) {
		cudaGetDeviceProperties(&devProp, device);
		printf("Device %d\n", device);
		printf("Compute capability      : %d.%d\n", devProp.major, devProp.minor);
		printf("Name                    : %s\n", devProp.name);
		printf("Total Global Memory     : %zu\n", devProp.totalGlobalMem);
		printf("Shared memory per block : %d\n", (int)devProp.sharedMemPerBlock);
		printf("Registers per block     : %d\n", devProp.regsPerBlock);
		printf("Warp size               : %d\n", devProp.warpSize);
		printf("Max threads per block   : (%d, %d, %d)\n", devProp.maxThreadsDim[0], devProp.maxThreadsDim[1], devProp.maxThreadsDim[2]);
		printf("Max block   : (%d, %d, %d)\n", devProp.maxGridSize[0], devProp.maxGridSize[1], devProp.maxGridSize[2]);
		printf("Total constant memory   : %d\n", (int)devProp.totalConstMem);
		printf("Multiprocessors count   : %d\n", devProp.multiProcessorCount);
	}
	return 0;
}

Found 1 devices
Device 0
Compute capability      : 7.5
Name                    : Tesla T4
Total Global Memory     : 15835398144
Shared memory per block : 49152
Registers per block     : 65536
Warp size               : 32
Max threads per block   : (1024, 1024, 64)
Max block   : (2147483647, 65535, 65535)
Total constant memory   : 65536
Multiprocessors count   : 40



In [ ]:
%%cu
#include <stdio.h>
#include <time.h>


void kernel(double* arr1, double* arr2, int n) {
    int i = 0;
    while(i < n) {
        arr1[i] = min(arr1[i], arr2[i]);
        i++;
    }
}


int main() {
    int n;
    n = 100000;
    double* arr1 = (double*)malloc(sizeof(double) * n);
    for(int i = 0; i < n; i++) {
        arr1[i] = i;
    }
    double* arr2 = (double*)malloc(sizeof(double) * n);
    for(int i = 0; i < n; i++) {
        arr2[i] = i;
    }
    clock_t start;
    start = clock();

    kernel(arr1, arr2, n);
    double time_used = ((double)(clock() - start)) / CLOCKS_PER_SEC;
    printf("%f\n", time_used * 1000);

    for(int i = 0; i < n; i++) {
        printf("%f ", arr1[i]);
    }
    printf("\n");

    free(arr1);
    free(arr2);

    return 0;
}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

